In [ ]:
import pandas as pd

CAP = 15
# CAP DATA AT 20 OR 15.



def word_count(string):
    return len(string.split(" "))

for i in ['train','valid']:
    for j in ['flickr','mscoco','visualgenome']:
        f = '/data/manifest/{}/{}_manifest.csv'.format(j,i)
        print(f)
        df = pd.read_csv(f)
        df['wordcount'] = df.caption.apply(word_count)
        print('max val', df.wordcount.max())
        print(df[df.wordcount==df.wordcount.max()]['caption'].values)
        
        cramp_df = df[df.wordcount<=CAP]
        print('{} out of {} is 30 len or below. thats {} percent'.format(len(cramp_df),len(df),len(cramp_df)/len(df)))
        print('=========\n')



In [ ]:
import torch

a = torch.arange(8).reshape((2,2,2)).float()

b = torch.nn.functional.normalize(a,p=2,dim=-1)
print(b)

c = []
for i in range((a.shape[1])):
    d = torch.nn.functional.normalize(a[:,i:i+1,:],p=2,dim=-1)
    c.append(d)
    
c = torch.cat(c,dim=1)
c==b
    


In [ ]:
import torch
learning_rate = 2
model = torch.nn.Linear(2, 1)
def get_learning_rate(epoch):
    if epoch < 10:
        return 1
    elif epoch < 15:
        return 0.5
    else:
        return 0.25
    
optimizer = torch.optim.Adam(
                model.parameters(), lr=learning_rate)
            
    
scheduler = torch.optim.lr_scheduler.LambdaLR(
                optimizer, lr_lambda=get_learning_rate)



lrs = []

for i in range(20):
    optimizer.step()
    lrs.append(optimizer.param_groups[0]["lr"])
#     print("Factor = ", round(0.65 ** i,3)," , Learning Rate = ",round(optimizer.param_groups[0]["lr"],3))
    scheduler.step()
    
print(lrs)

In [ ]:
import dask.dataframe as dd
import os
root_folder = '/data/image_manifest/flickr/train'
data = dd.read_parquet(os.path.join(root_folder),
                                    
                                    )  # this is lazy loading, its not actually loading into memory

In [ ]:
from data.text_tensor.dataset import VGTextDataset
from config.config import cfg
train_dataset = VGTextDataset('train', cfg)

In [ ]:
train_dataset[0]

In [ ]:

import os
from pathlib import Path
from clearml import Dataset, Task

DATA_PROJECT_NAME = "datasets/multimodal"


TASK_NAME = "visual_grounding_training_data_generation"

DATASET_ROOT = '/data/processed/'

OUTPUT_URL = "s3://experiment-logging/multimodal"


input_train_dataset_paths = []
input_valid_dataset_paths = []

args = {

    'num_captions': 5,
    'npartitions': 100,
    'batch_size': 4,
    'input_manifests': ['71a397b04b5d476bb5543306269e28af', ],
    'dataset_name': "VisualGround_FLICKR_manifest_100paritions_4_batch"
}


TEMP_PATH = '/tmp'
if not os.path.exists(TEMP_PATH):
    os.makedirs(TEMP_PATH)
for dataset_id in args['input_manifests']:
    dataset = Dataset.get(dataset_id=dataset_id)
    local_root_path = dataset.get_mutable_local_copy(
        os.path.join(TEMP_PATH, dataset_id))

    input_train_dataset_paths.append(os.path.join(
        local_root_path, 'train_manifest.csv'))

    valid_path = os.path.join(
        local_root_path, 'valid_manifest.csv')

    if os.path.exists(valid_path):
        input_valid_dataset_paths.append(valid_path)
print(input_train_dataset_paths)


In [ ]:
from PIL import Image
import os
grey = 'mscoco/COCO_train2014_000000095753.jpg'
image = Image.open(os.path.join(
            '/data/', grey))
rgbimg = Image.new("RGB", image.size)
rgbimg.paste(image)

In [ ]:
len(rgbimg.split())

In [ ]:
len(grey.split())

In [ ]:
rgbimg

In [1]:
import pandas as pd
import os
valid_data = pd.read_csv('/data/parquet/flickr_mscoco_visualgenome/valid/data.csv')

In [4]:
missing = []
for filename in valid_data.filename:
    if not check_exist(os.path.join('/data',filename)):
        missing.append(filename)
        print(filename)

In [3]:
def check_exist(f):
    return os.path.exists(f)

In [5]:
missing

[]

In [6]:
manifest_df = pd.read_csv('/data/manifests/flickr/valid_manifest.csv')

# manifest_df = manifest_df.groupby(
#             'filename').apply(lambda x: x.head(self.num_captions)).reset_index(drop=True)


In [9]:
num_captions = manifest_df.groupby('filename').filter(lambda x : len(x) >= 5)

In [12]:
num_captions.groupby('filename').apply(len).value_counts()

5    4482
dtype: int64